In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [3]:
from pathlib import Path

# Configurable parts
SOURCE = "dukascopy"
SYMBOL = "usdjpy"
START_DATE = "2020-01-01"
END_DATE = "2024-12-31"

# Build base name
BASE_NAME = f"{SOURCE}-{SYMBOL}-tick-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RAW_DIR = BASE_DIR / "raw"
RAW_FILE_PATH = RAW_DIR / f"{BASE_NAME}.csv"

In [4]:
def resample_to_time_bar(df, minutes: int):
    # Avoid unnecessary full copy if not modifying inplace
    df = df[['timestamp', 'askPrice', 'bidPrice', 'askVolume', 'bidVolume']]

    # Minimal temporary columns
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['mid'] = (df['askPrice'] + df['bidPrice']) / 2
    df['volume'] = df['askVolume'] + df['bidVolume']
    df['spread'] = df['askPrice'] - df['bidPrice']

    # Only keep what we need to resample
    df.set_index('timestamp', inplace=True)
    df = df[['mid', 'volume', 'spread']]

    # Perform resampling with reduced footprint
    ohlcv = df.resample(f'{minutes}min').agg({
        'mid': ['first', 'max', 'min', 'last'],
        'volume': 'sum',
        'spread': 'mean'
    })

    # Flatten column names
    ohlcv.columns = ['open', 'high', 'low', 'close', 'volume', 'spread']
    ohlcv = ohlcv.dropna(subset=['open'])  # Drop rows where no data existed in the interval

    ohlcv.reset_index(inplace=True)
    return ohlcv


In [5]:
def resample_to_dollar_bar(df, threshold: float):
    """
    Resample tick-level data into dollar bars using ask/bid quotes and volume.

    Parameters:
        df (pd.DataFrame): Must contain ['timestamp', 'askPrice', 'bidPrice', 'askVolume', 'bidVolume']
        threshold (float): Dollar value threshold to form one bar (e.g., 100_000)

    Returns:
        pd.DataFrame: Dollar bars with ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'spread']
    """
    df = df[['timestamp', 'askPrice', 'bidPrice', 'askVolume', 'bidVolume']].copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['mid'] = (df['askPrice'] + df['bidPrice']) / 2
    df['volume'] = df['askVolume'] + df['bidVolume']
    df['dollar'] = df['mid'] * df['volume']
    df['spread'] = df['askPrice'] - df['bidPrice']

    bars = []
    cum_dollar = 0.0
    bar = {'open': None, 'high': -float('inf'), 'low': float('inf'),
           'close': None, 'volume': 0.0, 'spread_sum': 0.0,
           'count': 0, 'start_time': None, 'end_time': None}

    for row in df.itertuples():
        price = row.mid
        vol = row.volume
        dol = row.dollar
        spread = row.spread
        ts = row.timestamp

        if bar['open'] is None:
            bar['open'] = price
            bar['start_time'] = ts

        bar['high'] = max(bar['high'], price)
        bar['low'] = min(bar['low'], price)
        bar['close'] = price
        bar['volume'] += vol
        bar['spread_sum'] += spread
        bar['count'] += 1
        bar['end_time'] = ts
        cum_dollar += dol

        if cum_dollar >= threshold:
            bars.append({
                'timestamp': bar['end_time'],
                'open': bar['open'],
                'high': bar['high'],
                'low': bar['low'],
                'close': bar['close'],
                'volume': bar['volume'],
                'spread': bar['spread_sum'] / bar['count'] if bar['count'] > 0 else None,
            })
            # Reset bar
            cum_dollar = 0.0
            bar = {'open': None, 'high': -float('inf'), 'low': float('inf'),
                   'close': None, 'volume': 0.0, 'spread_sum': 0.0,
                   'count': 0, 'start_time': None, 'end_time': None}

    return pd.DataFrame(bars)


In [6]:
import pandas as pd
import numpy as np

def check_time_continuity(ohlcv_df, minutes=60):
    # Ensure timestamp is in datetime format
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'])

    # Set timestamp as index for easier continuity check
    ohlcv_df.set_index('timestamp', inplace=True)

    # Generate a full datetime index at 1-minute intervals from the first to the last timestamp
    full_time_index = pd.date_range(start=ohlcv_df.index.min(), end=ohlcv_df.index.max(), freq=f"{minutes}min")

    # Find missing timestamps (gaps in the data)
    missing_timestamps = full_time_index.difference(ohlcv_df.index)

    # Print missing statistics
    missing_count = len(missing_timestamps)
    total_count = len(full_time_index)
    missing_percentage = (missing_count / total_count) * 100
    
    print(f"Total data points: {total_count}")
    print(f"Missing data points: {missing_count}")
    print(f"Percentage of missing data: {missing_percentage:.2f}%")
    print(f"First missing timestamp: {missing_timestamps.min() if missing_timestamps.size > 0 else 'N/A'}")
    print(f"Last missing timestamp: {missing_timestamps.max() if missing_timestamps.size > 0 else 'N/A'}")
    ohlcv_df.reset_index(inplace=True)
    return missing_timestamps



In [7]:
df = pd.read_csv(RAW_FILE_PATH)

In [8]:
df.shape

(163024077, 5)

In [9]:
df.head()

,timestamp,askPrice,bidPrice,askVolume,bidVolume
0,1577916000219,108.786,108.730,750.0,750.000000
1,1577916000433,108.790,108.730,750.0,750.000000
2,1577916023533,108.790,108.729,750.0,750.000000
3,1577916028663,108.790,108.728,750.0,1309.999943
4,1577916041516,108.791,108.728,750.0,560.000002


### Resampling 1minute time bar

In [ ]:
MINUTES=1
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
RESAMPLED_DIR = BASE_DIR / "resampled"
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"

In [12]:
df_1m = resample_to_time_bar(df, minutes=MINUTES)
df_1m

,timestamp,open,high,low,close,volume,spread
0,2020-01-01 22:00:00,108.7580,108.7600,108.7580,108.7590,9179.999948,0.060667
1,2020-01-01 22:01:00,108.7570,108.7590,108.7495,108.7495,13300.000012,0.060333
2,2020-01-01 22:02:00,108.7495,108.7535,108.7495,108.7535,4500.000000,0.058667
3,2020-01-01 22:03:00,108.7540,108.7555,108.7535,108.7555,10490.000010,0.059857
4,2020-01-01 22:04:00,108.7575,108.7650,108.7555,108.7650,11600.000024,0.055100
...,...,...,...,...,...,...,...
1860635,2024-12-30 23:55:00,157.0100,157.0110,157.0075,157.0080,206939.997196,0.011818
1860636,2024-12-30 23:56:00,157.0075,157.0090,157.0075,157.0075,127200.000405,0.010920
1860637,2024-12-30 23:57:00,157.0035,157.0095,156.9900,157.0070,234090.000987,0.007978
1860638,2024-12-30 23:58:00,157.0080,157.0080,157.0000,157.0015,317940.000176,0.008533


In [13]:
df_1m.to_pickle(RESAMPLED_FILE_PATH)

### Resampling 5minutes time bar

In [14]:
MINUTES=5
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
RESAMPLED_DIR = BASE_DIR / "resampled"
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"

In [15]:
df_5m = resample_to_time_bar(df, minutes=MINUTES)
df_5m

,timestamp,open,high,low,close,volume,spread
0,2020-01-01 22:00:00,108.7580,108.7650,108.7495,108.7650,4.907000e+04,0.058657
1,2020-01-01 22:05:00,108.7700,108.7700,108.7435,108.7460,2.120890e+06,0.024352
2,2020-01-01 22:10:00,108.7455,108.7600,108.6495,108.7450,6.706100e+05,0.026850
3,2020-01-01 22:15:00,108.7450,108.7480,108.6290,108.6910,2.436900e+05,0.067479
4,2020-01-01 22:20:00,108.6925,108.7210,108.6925,108.7210,8.848000e+04,0.048647
...,...,...,...,...,...,...,...
374072,2024-12-30 23:35:00,157.0160,157.0160,156.9610,157.0075,1.602750e+06,0.011005
374073,2024-12-30 23:40:00,157.0075,157.0345,157.0050,157.0335,1.957770e+06,0.014153
374074,2024-12-30 23:45:00,157.0330,157.0410,156.9830,156.9955,9.198600e+05,0.009822
374075,2024-12-30 23:50:00,156.9950,157.0125,156.9785,157.0125,4.284900e+05,0.008966


In [13]:
df_5m.to_pickle(RESAMPLED_FILE_PATH)

### Resampling 15minutes time bar

In [ ]:
MINUTES=15
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
RESAMPLED_DIR = BASE_DIR / "resampled"
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"

In [12]:
df_15m = resample_to_time_bar(df, minutes=MINUTES)
df_15m

,timestamp,open,high,low,close,volume,spread
0,2020-01-01 22:00:00,108.7580,108.7600,108.7580,108.7590,9179.999948,0.060667
1,2020-01-01 22:01:00,108.7570,108.7590,108.7495,108.7495,13300.000012,0.060333
2,2020-01-01 22:02:00,108.7495,108.7535,108.7495,108.7535,4500.000000,0.058667
3,2020-01-01 22:03:00,108.7540,108.7555,108.7535,108.7555,10490.000010,0.059857
4,2020-01-01 22:04:00,108.7575,108.7650,108.7555,108.7650,11600.000024,0.055100
...,...,...,...,...,...,...,...
1860635,2024-12-30 23:55:00,157.0100,157.0110,157.0075,157.0080,206939.997196,0.011818
1860636,2024-12-30 23:56:00,157.0075,157.0090,157.0075,157.0075,127200.000405,0.010920
1860637,2024-12-30 23:57:00,157.0035,157.0095,156.9900,157.0070,234090.000987,0.007978
1860638,2024-12-30 23:58:00,157.0080,157.0080,157.0000,157.0015,317940.000176,0.008533


In [13]:
df_15m.to_pickle(RESAMPLED_FILE_PATH)

### Resampling 60minutes time bar

In [ ]:
MINUTES=60
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
RESAMPLED_DIR = BASE_DIR / "resampled"
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"

In [12]:
df_60m = resample_to_time_bar(df, minutes=MINUTES)
df_60m

,timestamp,open,high,low,close,volume,spread
0,2020-01-01 22:00:00,108.7580,108.7600,108.7580,108.7590,9179.999948,0.060667
1,2020-01-01 22:01:00,108.7570,108.7590,108.7495,108.7495,13300.000012,0.060333
2,2020-01-01 22:02:00,108.7495,108.7535,108.7495,108.7535,4500.000000,0.058667
3,2020-01-01 22:03:00,108.7540,108.7555,108.7535,108.7555,10490.000010,0.059857
4,2020-01-01 22:04:00,108.7575,108.7650,108.7555,108.7650,11600.000024,0.055100
...,...,...,...,...,...,...,...
1860635,2024-12-30 23:55:00,157.0100,157.0110,157.0075,157.0080,206939.997196,0.011818
1860636,2024-12-30 23:56:00,157.0075,157.0090,157.0075,157.0075,127200.000405,0.010920
1860637,2024-12-30 23:57:00,157.0035,157.0095,156.9900,157.0070,234090.000987,0.007978
1860638,2024-12-30 23:58:00,157.0080,157.0080,157.0000,157.0015,317940.000176,0.008533


In [13]:
df_60m.to_pickle(RESAMPLED_FILE_PATH)